In [1]:
import os 
os.chdir('../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
from sklearn.linear_model import LogisticRegression

import KY_utils.KY_interpretable_functions as interpret
import KY_utils.KY_RiskSLIM as slim
import KY_utils.KY_stumps as stumps

# restore saved variables
%store -r summary_property2_ky_inter_model

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\kentucky\ky_fl_combined\KY Model


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### EBM & CART

In [2]:
KY_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_cleaned")
FL_data = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_cleaned")

## split x and y 
KY_X = KY_data.loc[:,:'current_violence20']
KY_Y = KY_data['property_two_year'].values
FL_X = FL_data.loc[:,:'current_violence20']
FL_Y = FL_data['property_two_year'].values

In [3]:
#### CART
depth = [6,7,8]
cart_summary = interpret.CART(KY_X, KY_Y, FL_X, FL_Y, depth, seed=816)

### EBM
estimators = [60]
depth = [2]
learning_rate = [0.5]
ebm_summary = interpret.EBM(KY_X, KY_Y, FL_X, FL_Y, learning_rate, depth, estimators, seed=816)

In [4]:
cart_summary['FL_score'],  ebm_summary['FL_score']

(0.6400032120774111, 0.6914012687705774)

### Lasso Stumps

In [5]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['property_two_year'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['property_two_year'].values

## columns 
cols = KY_X.columns

In [6]:
c_grid = {'C':[0.001, 0.002]}
stumps_summary = stumps.stump_cv(KY_X, KY_Y, FL_X, FL_Y, cols, c_grid, seed=816)

In [7]:
stumps_summary

{'best_auc': 0.8005695699225657,
 'best_params': {'C': 0.002},
 'number of coefs': 30,
 'dictionary': {'sex1': 0.18475123050975015,
  'age_at_current_charge18.0': 0.12142106875086126,
  'age_at_current_charge19.0': 0.09566811041918913,
  'age_at_current_charge20.0': 0.1151321764608373,
  'age_at_current_charge21.0': 0.0037087540955754034,
  'age_at_current_charge22.0': 0.04825812246953899,
  'age_at_current_charge25.0': 0.04311629602902808,
  'age_at_current_charge28.0': 0.07929467575978402,
  'age_at_current_charge32.0': 0.13758497401638073,
  'age_at_current_charge34.0': 0.035033536266763955,
  'age_at_current_charge37.0': 0.06636760928696994,
  'age_at_current_charge40.0': 0.3004042251552612,
  'age_at_current_charge45.0': 0.09853454969803953,
  'age_at_current_charge48.0': 0.030265736792014913,
  'age_at_current_charge70.0': -0.5715906684526597,
  'p_arrest1': -0.5048846544224944,
  'p_arrest2': 0.5847208956152565,
  'p_arrest3': 0.16273491340148827,
  'p_arrest4': 0.17352629592697

### RiskSLIM

In [8]:
KY_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/KY_stumps")
FL_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/psa-analysis - test2/kentucky/ky_fl_combined/data preparation/FL_stumps")

## split x and y 
KY_X = KY_stumps.loc[:,:'current_violence201']
KY_Y = KY_stumps['property_two_year'].values
FL_X = FL_stumps.loc[:,:'current_violence201']
FL_Y = FL_stumps['property_two_year'].values

## columns 
cols = KY_X.columns

In [9]:
## train on best param chosen by Lasso Stumps from above
lasso = LogisticRegression(class_weight='balanced', solver='liblinear', penalty='l1', C=0.0008, random_state=816).fit(KY_X, KY_Y)
selected_features = cols[lasso.coef_[0] != 0].tolist()
len(selected_features)

21

In [10]:
### Subset features
sub_FL_X = FL_stumps.loc[:, selected_features]
sub_KY_X = KY_stumps.loc[:, selected_features]
sub_KY_X.insert(0, '(Intercept)', 1)

In [11]:
riskslim_summary = slim.risk_cv(sub_KY_X, KY_Y, sub_FL_X, FL_Y, 
                                y_label = 'property_two_year', 
                                max_coef = 5, 
                                max_coef_number = 5, 
                                max_runtime=2000,
                                max_offset=100,
                                c=1e-3, 
                                seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
10/21/19 @ 03:29 PM | 263 rows in lookup table
10/21/19 @ 03:29 PM | ------------------------------------------------------------
10/21/19 @ 03:29 PM | runnning initialization procedure
10/21/19 @ 03:29 PM | ------------------------------------------------------------
10/21/19 @ 03:29 PM | CPA produced 2 cuts
10/21/19 @ 03:29 PM | running naive rounding on 13 solutions
10/21/19 @ 03:29 PM | best objective value: 0.0880
10/21/19 @ 03:29 PM | rounding produced 5 integer solutions
10/21/19 @ 03:29 PM | best objective value is 0.0927
10/21/19 @ 03:29 PM | running sequential rounding on 13 solutions
10/21/19 @ 03:29 PM | best objective value: 0.0880
10/21/19 @ 03:29 PM | sequential rounding produced 6 integer solutions
10/21/19 @ 03:29 PM | best objective value: 0.0868
10/21/19 @ 03:29 PM | polishing 11 solutions
10/21/19 @ 03:29 PM | best objective value: 0.0868
10/21/19 @ 03:29 PM | polishing produced 4 integer solutions
10/21/19 

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/21/19 @ 03:29 PM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.0868.
Tried aggregator 1 time.
Reduced MIP has 44 rows, 46 columns, and 129 nonzeros.
Reduced MIP has 21 binaries, 23 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.06 ticks)
Probing time = 0.00 sec. (0.02 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/21/19 @ 03:32 PM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.0885.
Tried aggregator 1 time.
Reduced MIP has 44 rows, 46 columns, and 129 nonzeros.
Reduced MIP has 21 binaries, 23 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.06 ticks)
Probing time = 0.00 sec. (0.02 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/21/19 @ 03:35 PM | adding 250 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.0891.
Tried aggregator 1 time.
Reduced MIP has 44 rows, 46 columns, and 129 nonzeros.
Reduced MIP has 21 binaries, 23 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.06 ticks)
Probing time = 0.00 sec. (0.02 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/21/19 @ 03:37 PM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.0867.
Tried aggregator 1 time.
Reduced MIP has 44 rows, 46 columns, and 129 nonzeros.
Reduced MIP has 21 binaries, 23 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.06 ticks)
Probing time = 0.00 sec. (0.02 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/21/19 @ 03:39 PM | adding 252 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.0864.
Tried aggregator 1 time.
Reduced MIP has 44 rows, 46 columns, and 129 nonzeros.
Reduced MIP has 21 binaries, 23 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.06 ticks)
Probing time = 0.00 sec. (0.02 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
10/21/19 @ 03:39 PM | adding 251 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.0864.
Tried aggregator 1 time.
Reduced MIP has 44 rows, 46 columns, and 129 nonzeros.
Reduced MIP has 21 binaries, 23 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.06 ticks)
Probing time = 0.00 sec. (0.02 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

In [12]:
riskslim_summary['FL_score'], np.mean(riskslim_summary['KY_train_score'])

(0.6348076768650125, 0.7383175262221165)

In [13]:
#### save results
summary_property2_ky_inter_model = {"CART": cart_summary,
                                   "EBM": ebm_summary,
                                   "Stumps": stumps_summary,
                                   "RiskSLIM": riskslim_summary}
%store summary_property2_ky_inter_model

Stored 'summary_property2_ky_inter_model' (dict)


In [14]:
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\KY Recidivism\\KY Results\\KY_FL\\KY Model\\interpretable\\Two Year\\"
results = [["Property",             
            round(cart_summary['FL_score'],3), 
            round(ebm_summary['FL_score'], 3), 
            round(stumps_summary['FL_score'],3), 
            round(riskslim_summary['FL_score'], 3)]]
with open(path + 'Two Year KY_inter_model.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)